In [1]:
import pandas as pd
import mysql.connector
import json

from urllib import request
from telegramBot import runBot
from time import sleep

bot = runBot.bot()
Url = 'https://api.elpais.com/ws/LoteriaNavidadPremiados?'

In [2]:
EUR = "€"
    
def API(tipoConsulta = 's=1'):

    return json.loads(request.urlopen(Url + tipoConsulta).read().decode('utf8').replace('info=', '').replace('busqueda=', ''))

def estado_sorteo():
    status = API()['status']  
    estado = None
    if status == 0:
        estado = 'El sorteo no ha comenzado.'
    elif status == 1:
        estado = 'El sorteo ha empezado. Lista de premios parcial.'
    elif status == 2:
        estado = 'Sorteo terminado. Lista de premios provisional.'
    elif status == 3:
        estado = 'Sorteo terminado. Lista de premios semioficial.'
    elif status == 4:
        estado = 'Sorteo terminado. Lista de premios oficial.'
    return estado 
    
def consultar(n): 
    premio = float(API('n=' + n)['premio'])
    return premio

conn = mysql.connector.connect(user = 'user' 
                               , password = 'pass' 
                               , host = 'host' 
                               , port = port 
                               , database = 'BbDd')

SqlStr = 'Select Numero, Importe, Descripcion from tabla where Anyo = (select max(Anyo) from tabla);'
df = pd.read_sql(SqlStr, conn)

df.head(2)

,Numero,Importe,Descripcion
0,97228,20.0,TuLotero
1,95321,20.0,ThePowerMBA


In [3]:
print(estado_sorteo())

Sorteo terminado. Lista de premios oficial.


In [6]:
informado, estado = 0, ''

while estado !='Sorteo terminado. Lista de premios oficial.':
    total_ganado = 0.0
    total_jugado = 0.0

    for i in range(len(df)):    
        ganado_decimo = consultar(str(df.Numero[i]))
        he_ganado = df.Importe[i] * ganado_decimo / 20
        total_ganado += he_ganado
        total_jugado += df.Importe[i]
        print(
            'Número: ' + '{0:05d}'.format(df.Numero[i]),
            'Jugado: ' + '{0:6.2f}'.format(df.Importe[i]) + " " + EUR,
            'Ganado: ' + '{0:10.2f}'.format(he_ganado) + " " + EUR,
            df.Descripcion[i],
            sep = '\t·\t',
        )
    estado = estado_sorteo()

    texto = estado + '\n'
    texto += '\nTotal Jugado:  <b>' + '{0:10.2f}</b>'.format(total_jugado) + EUR
    texto += '\nTotal Ganado:  <b>' + '{0:10.2f}</b>'.format(total_ganado) + EUR
    texto += '\nSaldo:              <b>' + '{0:10.2f}</b>'.format(total_ganado - total_jugado) + EUR

    if informado < total_ganado:
        bot.send(texto)
        informado = total_ganado
        
    sleep(60)
        
bot.send(texto)


Número: 97228	·	Jugado:  20.00 €	·	Ganado:       0.00 €	·	TuLotero
Número: 95321	·	Jugado:  20.00 €	·	Ganado:       0.00 €	·	ThePowerMBA
Número: 85046	·	Jugado:  10.88 €	·	Ganado:       0.00 €	·	Vodafone
Número: 53778	·	Jugado:   4.00 €	·	Ganado:       0.00 €	·	Departamento
Número: 60384	·	Jugado:  20.00 €	·	Ganado:       0.00 €	·	ONO
Número: 56252	·	Jugado:  20.00 €	·	Ganado:       0.00 €	·	Cancio
Número: 58895	·	Jugado:  20.00 €	·	Ganado:       0.00 €	·	Michelin
Número: 96976	·	Jugado:  20.00 €	·	Ganado:       0.00 €	·	El Carmen
Número: 67150	·	Jugado:  20.00 €	·	Ganado:     120.00 €	·	Cortecam
Número: 08364	·	Jugado:  10.00 €	·	Ganado:       0.00 €	·	Residencia Anselma
Número: 04671	·	Jugado:  10.00 €	·	Ganado:       0.00 €	·	Madrid
Número: 34230	·	Jugado:  20.00 €	·	Ganado:      20.00 €	·	Regalo Pedro
Número: 34034	·	Jugado:  20.00 €	·	Ganado:       0.00 €	·	Regalo Mama
Número: 99182	·	Jugado:   2.00 €	·	Ganado:       0.00 €	·	Karate
Número: 24345	·	Jugado:  10.00 €	·	Ganado:      

In [5]:
consultar('83000')

120.0